In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Peptic_ulcer_disease"
cohort = "GSE60427"

# Input paths
in_trait_dir = "../../input/GEO/Peptic_ulcer_disease"
in_cohort_dir = "../../input/GEO/Peptic_ulcer_disease/GSE60427"

# Output paths
out_data_file = "../../output/preprocess/Peptic_ulcer_disease/GSE60427.csv"
out_gene_data_file = "../../output/preprocess/Peptic_ulcer_disease/gene_data/GSE60427.csv"
out_clinical_data_file = "../../output/preprocess/Peptic_ulcer_disease/clinical_data/GSE60427.csv"
json_path = "../../output/preprocess/Peptic_ulcer_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Toll-like receptor 10 in Helicobacter pylori infection"
!Series_summary	"Helicobacter pylori is a highly successful and important human pathogen that causes chronic gastritis, peptic ulcer diseases and gastric cancer. Innate immunity plays an important role of the primary defense against pathogens and epidemiological studies have suggested a role of toll-like receptor 1 (TLR1) in the risk of H. pylori acquisition. We performed microarray analysis of gastric mucosal biopsy specimens from H. pylori-positive and uninfected subjects; infection was associated with an ~15-fold up-regulation of TLR10 (p <0.001). Quantitative RT-PCR confirmed  TLR10 mRNA levels were increased 3-fold in H. pylori-infection (p <0.001) and immunohistochemistory using anti-TLR10 polyclonal antibodies showed increased TLR10 expression in gastric epithelial cells of infected individuals. In vitro experiments where H. pylori was co-cultured with NCI-N87 gastric cells showed sign

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
is_gene_available = True  # Based on the background information, this is a microarray study of gene expression

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From sample characteristics, we identify the following:
age_row = 2  # Age information is available
gender_row = 1  # Gender information is available

# Looking deeper at the background information:
# The study mentions "Helicobacter pylori... causes chronic gastritis, peptic ulcer diseases and gastric cancer"
# And there's a characteristic for "gastritis grade" which could potentially indicate peptic ulcer disease
# Let's use the gastritis grade as a proxy for peptic ulcer disease presence
trait_row = 4  # 'gastritis grade: normal', 'gastritis grade: mild', 'gastritis grade: severe', 'gastritis grade: IM'

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert gastritis grade to binary trait (0 = normal/no disease, 1 = has disease)."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    value = value.lower().strip()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value == 'normal':
        return 0  # No disease
    elif value in ['mild', 'severe', 'im']:  # IM = intestinal metaplasia, a pathological condition
        return 1  # Has disease
    else:
        return None

def convert_age(value):
    """Convert age string to integer value."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    value = value.strip()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender string to binary (0 = female, 1 = male)."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    value = value.lower().strip()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value == 'f':
        return 0
    elif value == 'm':
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # This mimics the structure expected by geo_select_clinical_features
    sample_characteristics = {
        0: ['tissue: gastric epithelium'], 
        1: ['gender: F', 'gender: M'], 
        2: ['age: 50', 'age: 25', 'age: 66', 'age: 33', 'age: 27', 'age: 47', 'age: 38', 'age: 30', 'age: 28', 
            'age: 51', 'age: 40', 'age: 24', 'age: 53', 'age: 65', 'age: 34', 'age: 18', 'age: 61', 'age: 41', 
            'age: 64', 'age: 49', 'age: 42', 'age: 82', 'age: 73', 'age: 57', 'age: 46'], 
        3: ['country: Bhutan', 'country: Dominican Republic'], 
        4: ['gastritis grade: normal', 'gastritis grade: mild', 'gastritis grade: severe', 'gastritis grade: IM']
    }
    
    # Convert dictionary to DataFrame (ensuring it's in the format expected by geo_select_clinical_features)
    clinical_data = pd.DataFrame.from_dict(sample_characteristics, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Ensure directory exists before saving
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0, 50.0, 0.0], 1: [1.0, 25.0, 1.0], 2: [1.0, 66.0, nan], 3: [1.0, 33.0, nan], 4: [nan, 27.0, nan], 5: [nan, 47.0, nan], 6: [nan, 38.0, nan], 7: [nan, 30.0, nan], 8: [nan, 28.0, nan], 9: [nan, 51.0, nan], 10: [nan, 40.0, nan], 11: [nan, 24.0, nan], 12: [nan, 53.0, nan], 13: [nan, 65.0, nan], 14: [nan, 34.0, nan], 15: [nan, 18.0, nan], 16: [nan, 61.0, nan], 17: [nan, 41.0, nan], 18: [nan, 64.0, nan], 19: [nan, 49.0, nan], 20: [nan, 42.0, nan], 21: [nan, 82.0, nan], 22: [nan, 73.0, nan], 23: [nan, 57.0, nan], 24: [nan, 46.0, nan]}
Clinical data saved to ../../output/preprocess/Peptic_ulcer_disease/clinical_data/GSE60427.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['A_19_P00315452', 'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492',
       'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518',
       'A_19_P00315519', 'A_19_P00315524', 'A_19_P00315528', 'A_19_P00315529',
       'A_19_P00315538', 'A_19_P00315541', 'A_19_P00315543', 'A_19_P00315550',
       'A_19_P00315551', 'A_19_P00315554', 'A_19_P00315581', 'A_19_P00315583'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Agilent microarray probe IDs (A_19_P format)
# These are not standard human gene symbols and would need to be mapped to gene symbols
# They follow the Agilent probe format (A_19_P followed by a number)

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4', 'BPI fold containing family A, member 3'], 'UNIGENE_ID': [nan, nan, 'Hs.642618', 'Hs.348522', 'Hs.360989'], 'ENSEMBL_ID': [nan, nan, 'ENST00000014930', 'ENST00000281830', 'ENST00000375454'], 'ACCESSION_STRING': [nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788', 'ref|NM_178466|ens|ENST00000375454|ens|ENST00000471233|tc

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# From the gene annotation preview, we see:
# - 'ID' contains probe identifiers like 'A_23_P117082' which match the format in gene_data
# - 'GENE_SYMBOL' contains the human gene symbols we need like 'HEBP1'

# 2. Get a gene mapping dataframe by extracting the ID and GENE_SYMBOL columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# Preview the mapping
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the result to verify the gene symbols
print("\nGene expression data preview (first few gene symbols):")
print(gene_data.index[:10])

# Normalize gene symbols to handle synonyms and consolidate redundant entries
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print the shape of the resulting gene data
print(f"\nShape of the gene expression data: {gene_data.shape}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview:
{'ID': ['A_23_P117082', 'A_33_P3246448', 'A_33_P3318220', 'A_33_P3236322', 'A_33_P3319925'], 'Gene': ['HEBP1', 'KCNE4', 'BPIFA3', 'LOC100129869', 'IRG1']}

Gene expression data preview (first few gene symbols):
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT',
       'A4GNT', 'AA06'],
      dtype='object', name='Gene')



Shape of the gene expression data: (19847, 32)


Gene expression data saved to ../../output/preprocess/Peptic_ulcer_disease/gene_data/GSE60427.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the previously normalized gene data and saved clinical data
normalized_gene_data = pd.read_csv(out_gene_data_file, index_col=0)
clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)

print(f"Loaded normalized gene data with shape: {normalized_gene_data.shape}")
print(f"Loaded clinical data with shape: {clinical_data.shape}")

# Transpose clinical data to have samples as columns (matching gene data)
clinical_data_T = clinical_data.T
print(f"Clinical data after transposition: {clinical_data_T.shape}")

# Check which samples are in both datasets
gene_samples = set(normalized_gene_data.columns)
clinical_samples = set(clinical_data_T.columns)
common_samples = gene_samples.intersection(clinical_samples)
print(f"Gene samples: {len(gene_samples)}, Clinical samples: {len(clinical_samples)}, Common: {len(common_samples)}")

# Filter to keep only common samples
if len(common_samples) > 0:
    clinical_data_filtered = clinical_data_T[list(common_samples)]
    normalized_gene_data_filtered = normalized_gene_data[list(common_samples)]
    
    # 2. Link the clinical and genetic data using geo_link_clinical_genetic_data
    linked_data = geo_link_clinical_genetic_data(clinical_data_filtered, normalized_gene_data_filtered)
    print(f"Linked data shape: {linked_data.shape}")
    
    # The trait column is the first row from clinical data (Peptic_ulcer_disease)
    trait_col = linked_data.columns[0]
    print(f"Using trait column: {trait_col}")
    
    # Print counts of each trait value to help with debugging
    if trait_col in linked_data.columns:
        print(f"Trait values count: {linked_data[trait_col].value_counts().to_dict()}")
    
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait_col)
    print(f"Shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait_col)
    
    # 5. Conduct quality check and save the cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True,
        is_biased=is_trait_biased, 
        df=unbiased_linked_data,
        note="Dataset contains gastritis grade information as a proxy for peptic ulcer disease."
    )
    
    # 6. If the data is usable, save it
    if is_usable:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        # Save the data
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Data quality check failed. The dataset is not suitable for association studies.")
else:
    print("No common samples between clinical and gene expression data.")
    # Create a record indicating the dataset is not usable
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True,
        is_biased=True, 
        df=pd.DataFrame(),
        note="No common samples between clinical and gene expression data."
    )
    print("Data quality check failed. The dataset is not suitable for association studies.")

Loaded normalized gene data with shape: (19847, 32)
Loaded clinical data with shape: (3, 25)
Clinical data after transposition: (25, 3)
Gene samples: 32, Clinical samples: 3, Common: 0
No common samples between clinical and gene expression data.
Abnormality detected in the cohort: GSE60427. Preprocessing failed.
Data quality check failed. The dataset is not suitable for association studies.
